In [5]:
import pandas as pd
import os
import sys 

In [6]:
# Subo dos niveles desde data_analysis -> src -> g5-backend
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [7]:
from src.paths import RUTA_TITULO
from src.paths import DIR_DATA_PROCESADA
#from src.helpers import leer_tsv_chunks
from src.helpers import DatasetManager
from src.helpers import DataCleaner

from src.paths import RUTA_PERSONAL
from src.paths import RUTA_CRITICAS
from src.paths import RUTA_PRINCIPALES
from src.paths import RUTA_NOMBRE

Muestro la cantidad y tipos de títulos encontrados

In [4]:
cleaner=DataCleaner(RUTA_TITULO, DIR_DATA_PROCESADA)

lista_tipos=cleaner.mostrarContenidoDeUnaColumna('titleType')

print("Tipos de títulos únicos encontrados:")
for tipo in lista_tipos:
    print(f"- {tipo}")

print(f'\n Cantidad total de tipos de títulos: {len(lista_tipos)}')

Tipos de títulos únicos encontrados:
- movie
- short
- tvEpisode
- tvMiniSeries
- tvMovie
- tvPilot
- tvSeries
- tvShort
- tvSpecial
- video
- videoGame

 Cantidad total de tipos de títulos: 11


Filtro 5 títulos de *short, tvMovie, tvShort, tvSpecial*

In [10]:
# Lista de tipos que queremos filtrar
tipos_buscados = ['short', 'tvMovie', 'tvShort', 'tvSpecial']

# Diccionario para almacenar los resultados de cada tipo
resultados = {tipo: [] for tipo in tipos_buscados}

# Leer chunks y filtrar
titulos_iterable = cleaner.leer_tsv_chunks(cleaner.ruta_archivo)

for chunk in titulos_iterable:
    for tipo in tipos_buscados:
        # Si aún no tenemos 5 registros de este tipo, buscamos más
        if len(resultados[tipo]) < 5:
            filtrados = chunk[chunk['titleType'] == tipo]
            # Tomamos solo los que faltan hasta llegar a 5
            resultados[tipo].extend(filtrados.head(5 - len(resultados[tipo])).to_dict('records'))
    
    # Verificar si ya tenemos 5 de cada tipo
    if all(len(registros) >= 5 for registros in resultados.values()):
        break

# Mostrar resultados con tabla alineada
for tipo in tipos_buscados:
    print(f"\n=== {tipo.upper()} ===")
    header = f"{'Título Original':<35} | {'Año Inicio':<10} | {'Duración':<8} | Géneros"
    print(header)
    print("-" * len(header))
    
    for registro in resultados[tipo]:
        titulo = registro['originalTitle'][:35]  # Limitar a 35 caracteres
        anio = registro['startYear']
        duracion = registro['runtimeMinutes']
        generos = registro['genres']
        print(f"{titulo:<35} | {anio:<10} | {duracion:<8} | {generos}")


=== SHORT ===
Título Original                     | Año Inicio | Duración | Géneros
---------------------------------------------------------------------
Carmencita                          | 1894       | 1        | Documentary,Short
Le clown et ses chiens              | 1892       | 5        | Animation,Short
Pauvre Pierrot                      | 1892       | 5        | Animation,Comedy,Romance
Un bon bock                         | 1892       | 12       | Animation,Short
Blacksmith Scene                    | 1893       | 1        | Short

=== TVMOVIE ===
Título Original                     | Año Inicio | Duración | Géneros
---------------------------------------------------------------------
Julius Caesar                       | 1938       | 101      | Drama,History
As You Like It                      | 1946       | \N       | Drama
A Midsummer Night's Dream           | 1946       | 150      | Drama,Fantasy
Hamlet Part 1                       | 1947       | 88       | Drama
The Merch

Filtramos sólo las películas.  
De todo el dataset, guarda solo películas y TV movies

In [6]:
# Tipos de películas que queremos mantener
tipos_peliculas = ['movie', 'tvMovie']

# Lista para ir acumulando los chunks filtrados
peliculas_filtradas = []

# Leer y filtrar por chunks
titulos_iterable = DatasetManager().leer_tsv_chunks(RUTA_TITULO)
for chunk in titulos_iterable:
    # Filtrar solo los tipos que queremos
    chunk_filtrado = chunk[chunk['titleType'].isin(tipos_peliculas)]
    peliculas_filtradas.append(chunk_filtrado)

# Concatenar todos los chunks filtrados en un único DataFrame
df_peliculas = pd.concat(peliculas_filtradas, ignore_index=True)

In [7]:
# Mostrar información general
print(f"Total de registros filtrados: {len(df_peliculas)}")
print("\nDistribución por tipo:")
print(df_peliculas['titleType'].value_counts())

print("\nPrimeras 5 películas de cada tipo:")
for tipo in tipos_peliculas:
    print(f"\n=== {tipo.upper()} ===")
    print(df_peliculas[df_peliculas['titleType'] == tipo].head()[['primaryTitle', 'startYear', 'runtimeMinutes', 'genres']])

Total de registros filtrados: 876594

Distribución por tipo:
titleType
movie      724491
tvMovie    152103
Name: count, dtype: int64

Primeras 5 películas de cada tipo:

=== MOVIE ===
                    primaryTitle startYear runtimeMinutes  \
0                     Miss Jerry      1894             45   
1  The Corbett-Fitzsimmons Fight      1897            100   
2                       Bohemios      1905            100   
3    The Story of the Kelly Gang      1906             70   
4               The Prodigal Son      1907             90   

                       genres  
0                     Romance  
1      Documentary,News,Sport  
2                          \N  
3  Action,Adventure,Biography  
4                       Drama  

=== TVMOVIE ===
                    primaryTitle startYear runtimeMinutes         genres
20302              Julius Caesar      1938            101  Drama,History
26711             As You Like It      1946             \N          Drama
27062  A Midsummer Ni

Función para análisis de valores nulos

In [8]:
# Función para contar valores nulos y '\N'
def contar_nulos(df, columna):
    nulos_N = df[df[columna] == '\\N'].shape[0]  # Contar '\N'
    nulos_nan = df[columna].isna().sum()         # Contar NaN
    total = nulos_N + nulos_nan
    porcentaje = (total / len(df)) * 100
    return {
        'Valores \\N': nulos_N,
        'Valores NaN': nulos_nan,
        'Total Nulos': total,
        'Porcentaje': f"{porcentaje:.2f}%"
    }

Recorre todas las columnas y muestra cuántos nulos hay en cada una.

In [9]:
# Analizar cada columna
print("Análisis de valores nulos por columna:")
print("-" * 50)

for columna in df_peliculas.columns:
    resultados = contar_nulos(df_peliculas, columna)
    print(f"\nColumna: {columna}")
    print(f"- Valores '\\N': {resultados['Valores \\N']:,}")
    print(f"- Valores NaN: {resultados['Valores NaN']:,}")
    print(f"- Total nulos: {resultados['Total Nulos']:,}")
    print(f"- Porcentaje: {resultados['Porcentaje']}")

# Mostrar también cuántos registros tienen al menos un valor nulo
registros_con_nulos = df_peliculas[df_peliculas.apply(lambda x: (x == '\\N').any() or x.isna().any(), axis=1)]
print(f"\nRegistros con al menos un valor nulo: {len(registros_con_nulos):,} ({(len(registros_con_nulos)/len(df_peliculas)*100):.2f}% del total)")

Análisis de valores nulos por columna:
--------------------------------------------------

Columna: tconst
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: titleType
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: primaryTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: originalTitle
- Valores '\N': 0
- Valores NaN: 3
- Total nulos: 3
- Porcentaje: 0.00%

Columna: isAdult
- Valores '\N': 0
- Valores NaN: 0
- Total nulos: 0
- Porcentaje: 0.00%

Columna: startYear
- Valores '\N': 111,390
- Valores NaN: 0
- Total nulos: 111,390
- Porcentaje: 12.71%

Columna: endYear
- Valores '\N': 876,594
- Valores NaN: 0
- Total nulos: 876,594
- Porcentaje: 100.00%

Columna: runtimeMinutes
- Valores '\N': 314,612
- Valores NaN: 0
- Total nulos: 314,612
- Porcentaje: 35.89%

Columna: genres
- Valores '\N': 90,763
- Valores NaN: 0
- Total nulos: 90,763
- Porcentaje: 10.35%

Registros con al menos un va

Limpieza de nulos.  
Se eliminan los registros donde falten datos en columnas clave.

In [10]:
# Columnas a verificar
columnas_a_filtrar = ['genres', 'primaryTitle', 'originalTitle', 'runtimeMinutes']

# Crear una máscara para filtrar
mascara = True  # Comenzamos con todos los registros
for columna in columnas_a_filtrar:
    # Filtrar tanto '\N' como NaN
    mascara = mascara & (df_peliculas[columna] != '\\N') & (~df_peliculas[columna].isna())

# Aplicar el filtro
df_peliculas_limpio = df_peliculas[mascara]

In [11]:
# Mostrar información sobre los registros filtrados
registros_eliminados = len(df_peliculas) - len(df_peliculas_limpio)
print(f"Registros originales: {len(df_peliculas):,}")
print(f"Registros después de eliminar nulos: {len(df_peliculas_limpio):,}")
print(f"Registros eliminados: {registros_eliminados:,} ({(registros_eliminados/len(df_peliculas)*100):.2f}%)")

print("\nDistribución por tipo después de la limpieza:")
print(df_peliculas_limpio['titleType'].value_counts())

print("\nMuestra de 5 registros limpios:")
df_peliculas_limpio[['primaryTitle', 'titleType', 'startYear', 'runtimeMinutes', 'genres']].head()

Registros originales: 876,594
Registros después de eliminar nulos: 530,367
Registros eliminados: 346,227 (39.50%)

Distribución por tipo después de la limpieza:
titleType
movie      431138
tvMovie     99229
Name: count, dtype: int64

Muestra de 5 registros limpios:


,primaryTitle,titleType,startYear,runtimeMinutes,genres
0,Miss Jerry,movie,1894,45,Romance
1,The Corbett-Fitzsimmons Fight,movie,1897,100,"Documentary,News,Sport"
3,The Story of the Kelly Gang,movie,1906,70,"Action,Adventure,Biography"
4,The Prodigal Son,movie,1907,90,Drama
8,The Fairylogue and Radio-Plays,movie,1908,120,"Adventure,Fantasy"


Filtra solo películas del año 2019 que no sean para adultos.

In [12]:
# Filtrar películas del 2019
df_peliculas_filtrado = df_peliculas_limpio[
    (df_peliculas_limpio['startYear'] == '2019') & 
    (df_peliculas_limpio['isAdult'] == 0)]

In [13]:
print(f"Total de películas del 2019: {len(df_peliculas_filtrado):,}")
print("\nDistribución por tipo:")
print(df_peliculas_filtrado['titleType'].value_counts())

print("\nEstadísticas de duración (en minutos):")
print(df_peliculas_filtrado['runtimeMinutes'].astype(float).describe())


Total de películas del 2019: 16,810

Distribución por tipo:
titleType
movie      14615
tvMovie     2195
Name: count, dtype: int64

Estadísticas de duración (en minutos):
count    16810.000000
mean        93.533314
std        514.964989
min          1.000000
25%         70.000000
50%         88.000000
75%        101.000000
max      59460.000000
Name: runtimeMinutes, dtype: float64


In [14]:
df_peliculas_filtrado = df_peliculas_filtrado.drop(columns=["isAdult"])   #, "endYear"

print("\nMuestra de películas del 2019:")
df_peliculas_filtrado.head(10)


Muestra de películas del 2019:


,tconst,titleType,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,genres
87763,tt0116991,movie,Mariette in Ecstasy,Mariette in Ecstasy,2019,\N,101,Drama
217353,tt0385887,movie,Motherless Brooklyn,Motherless Brooklyn,2019,\N,144,"Crime,Drama,Mystery"
233764,tt0437086,movie,Alita: Battle Angel,Alita: Battle Angel,2019,\N,122,"Action,Adventure,Sci-Fi"
235270,tt0441881,movie,Danger Close,Danger Close: The Battle of Long Tan,2019,\N,118,"Action,Drama,War"
237327,tt0448115,movie,Shazam!,Shazam!,2019,\N,132,"Action,Adventure,Comedy"
254146,tt0783640,movie,The Last Full Measure,The Last Full Measure,2019,\N,116,"Drama,War"
254769,tt0791072,movie,Enemigo,Enemigo,2019,\N,93,Drama
255734,tt0800325,movie,The Dirt,The Dirt,2019,\N,107,"Biography,Comedy,Drama"
256662,tt0810836,movie,Dirt Music,Dirt Music,2019,\N,105,"Crime,Drama,Romance"
259568,tt0837563,movie,Pet Sematary,Pet Sematary,2019,\N,100,"Horror,Mystery,Thriller"


In [15]:
DatasetManager().guardar_csv(df_peliculas_filtrado, "peliculas_2019.csv")

Archivo guardado: peliculas_2019.csv


## ACA SE HACE UN LIMPIEZA EN BASE A LOS IDS DE LAS PELICULAS.... hay que revisar

In [16]:
ids_peliculas = df_peliculas_filtrado['tconst']

In [17]:
# Función para filtrar chunks basado en los IDs
def filtrar_por_ids(chunk):
    return chunk[chunk['tconst'].isin(ids_peliculas)]

In [18]:
# Filtrar tabla de críticas
print("Filtrando críticas...")
chunks_filtrados = []
criticas_iterable = DatasetManager().leer_tsv_chunks(RUTA_CRITICAS)
for chunk in criticas_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_criticas_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()

Filtrando críticas...


In [19]:
# Filtrar tabla de personal
print("Filtrando personal...")
chunks_filtrados = []
personal_iterable = DatasetManager().leer_tsv_chunks(RUTA_PERSONAL)
for chunk in personal_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_personal_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()

Filtrando personal...


In [20]:
# Filtrar tabla de principales
print("Filtrando principales...")
chunks_filtrados = []
principales_iterable = DatasetManager().leer_tsv_chunks(RUTA_PRINCIPALES)
for chunk in principales_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_principales_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()

Filtrando principales...


In [21]:
#TODO: TIRA ERROR

# Filtrar tabla de nombres
print("Filtrando nombres...")
chunks_filtrados = []
nombres_iterable = DatasetManager().leer_tsv_chunks(RUTA_NOMBRE)
for chunk in nombres_iterable:
    filtrado = filtrar_por_ids(chunk)
    if not filtrado.empty:
        chunks_filtrados.append(filtrado)
df_nombres_filtrado = pd.concat(chunks_filtrados, ignore_index=True) if chunks_filtrados else pd.DataFrame()

Filtrando nombres...


KeyError: 'tconst'

In [22]:
df_personal_filtrado.head(10)

,tconst,directors,writers
0,tt0116991,nm0007037,nm0361069
1,tt0385887,nm0001570,"nm0001570,nm0504672"
2,tt0437086,nm0001675,"nm0000116,nm0436164,nm1738737"
3,tt0441881,nm0826541,"nm0064181,nm1792356,nm1764825,nm0838283,nm4870103"
4,tt0448115,nm2497546,"nm2849655,nm0501359,nm1791910,nm1304148"
5,tt0783640,nm0733149,nm0733149
6,tt0791072,nm0479586,nm0479586
7,tt0800325,nm0871860,"nm0929186,nm5287325,nm0005136,nm1016427,nm0624..."
8,tt0810836,nm0429963,"nm2113666,nm0936090"
9,tt0837563,"nm1556116,nm1552637","nm0000175,nm0338557,nm1161528"


In [23]:
DatasetManager().guardar_csv(df_criticas_filtrado, "criticas_2019.csv")
DatasetManager().guardar_csv(df_personal_filtrado, "personal_2019.csv")
#guardar_csv(df_nombres_filtrado, "nombres_2019.csv")
DatasetManager().guardar_csv(df_principales_filtrado, "principales_2019.csv")

Archivo guardado: criticas_2019.csv
Archivo guardado: personal_2019.csv
Archivo guardado: principales_2019.csv
